# MLP


## 用語解説

- MLPとは

    Multi Layer Perceptron の略で、日本語にすると多層パーセプトロン。

- パーセプトロン

- 人工ニューロン

- 活性化関数

資料の流れ

前回までで環境構築と深層学習のざっくりとした説明をした

今回からは、実際に深層学習に触れていこうと思う

まず今回は、基礎となる浅いニューラルネットワークについてみていく。
- パーセプトロンとは
- 重みとバイアス
- ニューラルネットワーク
- 活性化関数
- 誤差関数
- 最適化関数
    - 最急降下法
    - 確率的勾配法(SGD)
- 誤差逆伝播

### パーセプトロン
パーセプトロンとは1957年に考案されたアルゴリズムで、現在のディープラーニングの基礎になっている重要なものである。

パーセプトロンは複数の信号を受け取り、1つの信号を出力をする。ここでの信号とは、データや電流などの流れをもつものだとイメージすると良い。

簡単のため、入力が2つの場合のパーセプトロンを考える。パーセプトロンの入力をそれぞれ$x_1$,$x_2$とし、それらの流れやすさ。つまり、重みを$w_1$,$w_2$。出力を$y$とする

$$
y = \left\{
    \begin{array}{cc}
    0  &(w_1x_1 + w_2x_2 \le 0)\\
    1  &(w_1x_1 + w_2x_2 > 0)
    \end{array}
    \right.
$$

$y$の出力は0か1の2通りである。

![](./fig/03/01.jpeg)

In [ ]:
def perceptlon(x_1, x_2):
    w_1, w_2 = 1, 1 # 重みを変えると同じ入力でもyの値が変わる
    result = w_1*x_1 + w_2*x_2
    y = 0
    if result > 0:
        y = 1
    return y

In [ ]:
print(perceptlon(1,-1))

## MLP
全結合層のみで構成されているやつ。

![](./fig/03/02.png)

今回は、MLPを用いて、クラス分類をしていく。データセットはskleran が用意している 手書き文字を用いる

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.moel_selection import train_test_split
from tqdm import tqdm

In [ ]:
digits = load_digits()

image = digits.data
label = digits.target

# NumpyのndarrayをTensorに変換
image = torch.tensor(image, dtype=torch.float32)
label = torch.tensor(label, dtype=torch.int64)

# データセットを作成
dataset = TensorDataset(image, label)
# train に使う方は順番をランダムにしておく
# ミニバッチといって、データの塊を用意しておく
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset, batch_size=64, shuffle=False)

# ネットワーク定義
# 今回は、線形層が３層。活性化関数はReLUを用いる
net = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
)

In [ ]:
def eval_net(net, data_loader, device='cpu'):
    net.eval()
    labels = []
    label_preds = []
    for image, label in loader:
        image = image.to(device)
        label = label.to(device)
        with torch.no_grad():
            _, label_pred = net(image).max(1)
        labels.append(label)
        label_preds.append(label_pred)
    # ミニバッチを1つに集める
    labels = torch.cat(labels)
    label_preds = torch.cat(label_preds)
    # 予想精度を計算
    acc = (labels == label_preds).float().sum() / len(labels)
    print(acc.item())
    return acc.item()

In [ ]:
def train_net(net, train_loader, test_loader, optimizer=optim.Adam,
                loss_fn = nn.CrossEntropyLoss(), n_iter=10, device='cpu'):
    train_losses = []
    train_acc = []
    eval_acc = []
    optimizer =  optimizer(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.
        net.train()
        n = 0
        n_acc = 0
        # プログレスバーを出す
        for i, (image, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
            image = image.to(device)
            label = label.to(device)

            pred_label = net(image)
            
            loss = loss_fn(pred_label, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(image)
            # 確率が一番高いやつを計算
            _, label_pred = pred_label.max(1)
            n_acc += (label == label_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 訓練データの予測精度
        train_acc.append(n_acc / n)
        eval_acc.append(eval_net(net, test_loader, device))
        print(epoch, train_losses[-1], train_acc[-1], eval_acc[-1], flush=True)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
net.to(device)
train_net(net, train_loader, test_loader, n_iter=20, device=device)